In [44]:
# !pip3 install matplotlib
# !pip3 install tabula-py
# !pip3 install PyPDF2
# !pip3 install pdfminer.six
# !pip3 install PyMuPDF
# !pip3 install camelot-py
# !pip3 install --upgrade pip
# !pip3 install geopandas
# !pip3 install plotly
# !pip3 install pandas
# !pip3 install pdfplumber
# !pip3 install scipy
# !pip3 install --upgrade pandas
# !pip3 install arcgis
# !pip3 install json
!pip3 install gtfs_functions
!pip3 install keplergl

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.2/228.2 kB 4.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.3/353.3 kB 4.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.8/987.8 kB 6.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 14.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 10.1 MB/s eta 0:00:00
  Created wheel for utm: filename=utm-0.7.0-py3-

In [46]:
import pandas as pd
import numpy as np
import matplotlib as mpt
import os
# import PyPDF2
# import tabula
# import tabulate
# import pdfminer
import geopandas as gpd
import fiona
from glob import glob
import plotly.express as px
from shapely.geometry import Point, MultiPolygon
# import pdfplumber
import scipy
import json
import arcgis
from gtfs_functions import Feed
from gtfs_functions.gtfs_plots import map_gdf
import keplergl as kpl


In [47]:

def save_to_csv(df, output_folder, file_name):
    """AI is creating summary for save_to_csv

    Args:
        df (Pandas Data Frame): Contains data frame that needs to be converted and saved as csv
        output_folder (Str): Destination path where df needs to be saved
        file_name (Str): File name that df needs to be saved as
    """
    # Check if the output folder exists, if not, create it
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Define the output file path
    output_file_path = os.path.join(output_folder, file_name)
    
    # Save the DataFrame to CSV in the output folder
    df.to_csv(output_file_path, index=False)
    print(f"DataFrame saved to: {output_file_path}")


In [48]:
def convert_data (file_path):
    """
    Function to convert files of appropriate structure to csv format, creating point object with files that have lat and lon values

    Args:
        file_path (str): File Path to open and convert to csv file format
    """
    df = pd.read_csv(file_path)
    output_folder_path = os.getcwd()+"/Data Sets/Analysis Data/"
    # print(output_folder_path)
    output_file_name = file_path.split('/')[-1].split('.')[0] + ".csv"
    # output_destination = os.path.join(output_folder_path, output_file_name)
    # if 'lat' in str(df.columns) and 'lon' in str(df.columns):
    #     lat = ""
    #     lon = ""
    #     for col in df.columns:
    #         if str(col).endswith('lat'):
    #             lat = str(col)
    #         elif str(col).endswith('lon'):
    #             lon = str(col)
    #     df_geo = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[lat],df[lon]))
    #     if not os.path.exists(output_folder_path):
    #         os.make_dir(output_folder_path)
    #     # df_geo.drop(columns=[lat, lon]).to_csv(output_destination, index = False)
    #     final_df = df_geo.drop(columns=[lat, lon])
    #     save_to_csv(final_df, output_folder_path, output_file_name)
    #     print("{} saved in {}".format(output_file_name, output_folder_path))
    # else:
    # df.to_csv(output_destination, index=False)
    save_to_csv(df, output_folder_path, output_file_name)
    print("{} saved in {}".format(output_file_name, output_folder_path))

In [4]:
data_path = os.getcwd()+"/Data Sets/CT_GTFS/"
# ignore_file = ['agency.txt', "calendar.txt","calendar_dates.txt"]
convert_data(data_path+"shapes.txt")
# Loop through files in the folder
# for filename in os.listdir(data_path):
#     # print(filename)
#     file_path = data_path + filename
#     if os.path.isfile(file_path) and filename not in ignore_file:
#         try:
#             with open(file_path, 'r', encoding='utf-8') as file:
#                 # Assuming text files, read the content
#                 content = file.read()
#                 # Do something with the content
#                 convert_data(content)
#         except UnicodeDecodeError:
#             print("Error decoding file:", file_path)
#             continue

DataFrame saved to: /Users/sulemanbasit/Project/Calgary-Transit-Economic-Gap-Analysis/Data Sets/Analysis Data/shapes.csv
shapes.csv saved in /Users/sulemanbasit/Project/Calgary-Transit-Economic-Gap-Analysis/Data Sets/Analysis Data/


# Program To Calculate Demand Index Using Z-Method

In [5]:
community_data_path = os.getcwd()+"/Data Sets/Analysis Data/Community Profiles Compiled.xlsx"

In [6]:
comm_df = pd.read_excel(community_data_path)
# comm_df.head()

In [7]:
comm_df.head()

,Community Name,Population in private households,0 to 14 years,65 to 84 years,85 years and over,Population aged 15 years and over in private households,In the labor force,Employed,Self-Employed,Unemployed,...,15 to 29 minutes,30 to 44 minutes,45 to 59 minutes,Employed labour force aged 15 years and over in private households3,Between 5 a.m. and 5:59 a.m.,Between 6 a.m. and 6:59 a.m.,Between 7 a.m. and 7:59 a.m.,Between 8 a.m. and 8:59 a.m.,Between 9 a.m. and 11:59 a.m.,Between 12 p.m. and 4:59 a.m.
0,Abbeydale,5925,1200,605,50,4725,3115,2650,295,470,...,1260,465,105,2335.0,290,420,530,300,275,525
1,Acadia,9915,1280,1555,290,8635,5765,4985,775,780,...,1675,565,160,3840.0,245,735,1050,795,495,525
2,Albert Park/Radisson Heights,6740,1160,675,80,5580,3415,2905,400,510,...,1195,435,140,2430.0,235,500,535,280,360,525
3,Altadore,7290,1475,565,70,5810,4480,4105,900,375,...,1155,195,40,2245.0,65,330,770,515,350,215
4,Applewood Park,6830,1155,700,40,5680,3775,3180,425,595,...,1535,500,150,2790.0,295,550,490,370,465,620


In [8]:
output_folder_path = os.getcwd()+"/Data Sets/Analysis Data"

In [9]:
#Z-Score method manually
pop_mean = comm_df['Population in private households'].mean()
pop_std = comm_df['Population in private households'].std()


medinc_mean = comm_df["Median household income of private households"].mean()
medinc_std = comm_df["Median household income of private households"].std()

low_mean = comm_df['Population in private households to whom low income concepts are applicable (Number in low income)'].mean()
low_std = comm_df['Population in private households to whom low income concepts are applicable (Number in low income)'].std()

trans_mean = ((comm_df['Employed']/comm_df["Employed labour force aged 15 years and over in private households"])*comm_df['Public transit']).mean()
trans_std = ((comm_df['Employed']/comm_df["Employed labour force aged 15 years and over in private households"])*comm_df['Public transit']).std()

rent_mean = (comm_df['Per cent households with income spending 30% or more total income on shelter (Renter)']*comm_df['Private households with total income greater than zero (Renter)']).mean()
rent_std = (comm_df['Per cent households with income spending 30% or more total income on shelter (Renter)']*comm_df['Private households with total income greater than zero (Renter)']).std()

seniors_mean = comm_df['65 to 84 years'].mean()
seniors_std = comm_df['65 to 84 years'].std()

In [10]:
row, col = comm_df.shape
z_rows_to_append = []
for c in range(row):
    community = comm_df['Community Name'][c]
    
    pop = comm_df['Population in private households'][c]
    # pop_ss = (pop-pop_max)/(pop_max-pop_min)
    z_pop_score = (pop-pop_mean)/pop_std
    
    med = comm_df['Median household income of private households'][c]
    # med_ss = (med-medinc_max)/(medinc_max-medinc_min)
    z_med_score = (med-medinc_mean)/medinc_std
    
    trans = ((comm_df['Employed']/comm_df["Employed labour force aged 15 years and over in private households"])*comm_df['Public transit'])[c]
    # trans_ss = (trans-trans_max)/(trans_max-trans_min)
    z_trans_score = (trans-trans_mean)/trans_std
    
    rent = (comm_df['Per cent households with income spending 30% or more total income on shelter (Renter)']*comm_df['Private households with total income greater than zero (Renter)'])[c]
    # rent_ss = (rent-rent_max)/(rent_max-rent_min)
    z_rent_score = (rent-rent_mean)/rent_std
    
    low = comm_df['Population in private households to whom low income concepts are applicable (Number in low income)'][c]
    # low_ss = (low-low_max)/(low_max-low_min)
    z_low_score = (low-low_mean)/low_std

    sen = comm_df['65 to 84 years'][c]
    z_seniors_score = (sen-seniors_mean)/seniors_std

    z_result = z_trans_score+z_rent_score+z_low_score-z_med_score+z_seniors_score
    # z_result = z_trans_score+z_rent_score+z_low_score+z_seniors_score

    column_df = {"Community Name": community, "Z Score": z_result}
    columns_df = {"Community Name": community, "Low Income Index": z_low_score, "Seniors Index": z_seniors_score, "Public Transit Index": z_trans_score, "Rent Index": z_rent_score, "Median Income": z_med_score}
    z_rows_to_append.append(column_df)


Z_index_df = pd.DataFrame(z_rows_to_append)

In [11]:
Z_index_df.head()

,Community Name,Z Score
0,Abbeydale,0.084462
1,Acadia,5.514759
2,Albert Park/Radisson Heights,3.648275
3,Altadore,-1.806846
4,Applewood Park,1.123514


In [12]:
z_max = Z_index_df['Z Score'].max()
z_min = Z_index_df['Z Score'].min()

In [13]:
print(z_max)
print(z_min)

28.524727731817897
-10.6026423782651


In [14]:
row, col = Z_index_df.shape
z_ss_rows = []
for c in range(row):
    community = Z_index_df["Community Name"][c]
    z_score = Z_index_df['Z Score'][c]

    z_ss_result = (z_score-z_min)/(z_max-z_min)

    column_df = {"Community Name": community, "Z Score": z_ss_result}
    z_ss_rows.append(column_df)
    
z_ss_index_df = pd.DataFrame(z_ss_rows)

In [15]:
z_ss_index_df.head()

,Community Name,Z Score
0,Abbeydale,0.273136
1,Acadia,0.411921
2,Albert Park/Radisson Heights,0.364219
3,Altadore,0.224799
4,Applewood Park,0.299692


In [16]:
save_to_csv(z_ss_index_df, output_folder_path, 'Z Standardized Demand Index.csv')

DataFrame saved to: /Users/sulemanbasit/PycharmProjects/Calgary-Transit-Economic-Gap-Analysis/Data Sets/Analysis Data/Z Standardized Demand Index.csv


# Program To Visualize ArcGis Data

In [37]:
gis_link = "https://ucalgary.maps.arcgis.com"
gis_user = "sbasit_ucalgary"
gis_pass = "Sa5nar9("

In [38]:
my_gis = arcgis.GIS(gis_link,gis_user,gis_pass)

NameError: name 'arcgis' is not defined

In [40]:
help(arcgis.GIS)

Help on class GIS in module arcgis.gis:

class GIS(builtins.object)
 |  GIS(url: 'Optional[str]' = None, username: 'Optional[str]' = None, password: 'Optional[str]' = None, key_file: 'Optional[str]' = None, cert_file: 'Optional[str]' = None, verify_cert: 'bool' = True, set_active: 'bool' = True, client_id: 'Optional[str]' = None, profile: 'Optional[str]' = None, **kwargs)
 |  
 |  .. _gis:
 |  
 |  The ``GIS`` class is representative of a single ArcGIS Online organization or an ArcGIS Enterprise deployment.
 |  The ``GIS`` object provides helper objects to manage (search, create, retrieve) GIS resources such as content, users,
 |  and groups.
 |  
 |  Additionally, the ``GIS`` object has properties to query its state, which is accessible using the properties attribute.
 |  
 |  .. note::
 |  
 |      The ``GIS`` provides a mapping widget that can be used in the Jupyter Notebook environment for visualizing GIS content
 |      as well as the results of your analysis. To create a new map,

In [41]:
my_gis = arcgis.GIS(api_key = "AAPK049cf86d96f4462ba33f8b661792d2f0z748Z0jNex7rIOx45MLoRtWEUFk5CElu0-Obz_3bxUwj9_Q8MI398NcMkS9xWe5F")

In [42]:
map = my_gis.map("d343d4c8db7343368ad524174e793da1")

In [43]:
map

MapView(layout=Layout(height='400px', width='100%'))

# Join Multipolygon Data To Profile

In [22]:
folder_path = os.getcwd()+"/Data Sets/Analysis Data/"
comm_file = "Community Profiles Compiled.xlsx"
bound_file = "Boundraries_Data_by_Community .csv"
z_index_file = "Z Standardized Demand Index.csv"

In [23]:
comm_data = pd.read_excel(folder_path+comm_file)
comm_bound = gpd.read_file(folder_path+bound_file)
z_index = pd.read_csv(folder_path+z_index_file)

In [31]:
comm_bound_lower = comm_bound.copy()
comm_bound_lower['NAME'] = comm_bound_lower['NAME'].str.lower()
comm_data_lower = comm_data.copy()
comm_data_lower['Community Name'] = comm_data_lower['Community Name'].str.lower()
comm_z_lower = z_index.copy()
comm_z_lower["Community Name"] = comm_z_lower['Community Name'].str.lower()

# Merge the DataFrames based on the lower case names
# merged_data = pd.merge(comm_data_lower, comm_bound_lower, left_on='Community Name', right_on='NAME', how='inner')
merged_data = pd.merge(comm_z_lower, comm_bound_lower, how='left', left_on='Community Name', right_on='NAME')

# Convert back to original case if needed
merged_data['NAME'] = comm_bound['NAME']  # Assuming you want to keep the original case for boundary names
merged_data['Community Name'] = comm_data['Community Name']  # Assuming you want to keep the original case for community names

# Now merged_data contains the rows where the names match, regardless of case

In [32]:
merged_data.drop(columns=["geometry"], inplace=True)
merged_data.drop(columns=["NAME"], inplace=True)

In [35]:
merged_data.shape

(202, 3)

In [36]:
save_to_csv(merged_data, folder_path, "Community_Profile_Z_Index.csv")

DataFrame saved to: /Users/sulemanbasit/PycharmProjects/Calgary-Transit-Economic-Gap-Analysis/Data Sets/Analysis Data/Community_Profile_Z_Index.csv


In [30]:
type(comm_bound['multipolygon'][1])

str

# Calculate Supply Frequency

In [72]:
gtfs_path = os.getcwd()+"/Data Sets/CT_GTFS.zip"
feed = Feed(gtfs_path)

In [73]:
routes = feed.routes
trips = feed.trips
stops = feed.stops
stop_times = feed.stop_times
shapes = feed.shapes

INFO:root:Reading "routes.txt".
INFO:root:accessing trips
INFO:root:Start date is None. You should either specify a start date or set busiest_date to True.
INFO:root:Reading "trips.txt".
INFO:root:Reading "calendar.txt".
INFO:root:Reading "calendar_dates.txt".
INFO:root:The busiest date/s of this feed or your selected date range is/are:  ['2024-02-19'] with 17641 trips.
INFO:root:In the case that more than one busiest date was found, the first one will be considered.
INFO:root:In this case is 2024-02-19.
INFO:root:Reading "stop_times.txt".
INFO:root:_trips is defined in stop_times
INFO:root:Reading "stops.txt".
INFO:root:computing patterns
/Users/sulemanbasit/Library/Python/3.9/lib/python/site-packages/gtfs_functions/gtfs_functions.py:380: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

In [177]:
feed = Feed(gtfs_path,start_date="2024-02-17", end_date="2024-02-17")
stop_freq = feed.stops_freq

INFO:root:Reading "stop_times.txt".
INFO:root:get trips in stop_times
INFO:root:accessing trips
INFO:root:Reading "routes.txt".
INFO:root:Reading "trips.txt".
INFO:root:Reading "calendar.txt".
INFO:root:Reading "calendar_dates.txt".
INFO:root:The busiest date/s of this feed or your selected date range is/are:  ['2024-02-17'] with 6826 trips.
INFO:root:In the case that more than one busiest date was found, the first one will be considered.
INFO:root:In this case is 2024-02-17.
INFO:root:Reading "stop_times.txt".
INFO:root:_trips is defined in stop_times
INFO:root:Reading "stops.txt".
INFO:root:computing patterns
/Users/sulemanbasit/Library/Python/3.9/lib/python/site-packages/gtfs_functions/gtfs_functions.py:380: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

In [196]:
stop_list = [3641, 3816, 3817, 3960, 4155, 4934, 6801, 6802, 6803, 6804, 6805, 6806, 6807, 6815, 6816, 6817, 6818, 6819, 6820, 6822, 6823, 6824, 6825, 6827, 6828, 6829, 6830, 6831, 8556, 8557, 8558, 8559, 8560, 8561, 8562, 8563, 8564, 8565, 8566, 9261, 9262, 9263, 9264, 9387, 9390, 9391, 9392, 3627, 3628, 3629, 3630, 3631, 3632, 3633, 3634, 3635, 3636, 3637, 3638, 3639, 6747, 6808, 6809, 6810, 6811, 6812, 6813, 6814, 6825, 6831, 8567, 8568, 8569, 8570, 8571, 8572, 9385, 9396, 9781, 9896, 9897, 2350, 2376, 2793, 2794, 2795, 2796, 2797, 2798, 2799, 2800, 2801, 2802, 2803, 3495, 5701, 5703, 5718, 5730, 5731, 5732, 5734, 5789, 6048, 6724, 6846, 7281, 7759, 7762, 7763, 7764, 7881, 7892, 8157, 8161, 8162, 8163, 8215, 9400, 9407, 9409, 9430, 9439, 9617, 3368, 3370, 3371, 3372, 3380, 3381, 3382, 3383, 3384, 3751, 3761, 4567, 4568, 4582, 4583, 5147, 5385, 5630, 5752, 5754, 5767, 5769, 6099, 6110, 6111, 6586, 6982, 8038, 8368, 9174, 9178, 9828, 2469, 2486, 2487, 2488, 2489, 2498, 3371, 3531, 3744, 3745, 3746, 3747, 3748, 3755, 3756, 3764, 3765, 3766, 3767, 3870, 5762, 5763, 6119, 6982, 9453, 9935, 2178, 2180, 2234, 2299, 2788, 3387, 3388, 3389, 3390, 3391, 3392, 3886, 4352, 5057, 7380, 8271, 8653, 8832, 9802, 9805, 9807, 9808, 9809, 9810, 9815, 9816, 9817, 9818, 9820, 9821]
stop_list_str = [str(x) for x in stop_list]
stop_freq.loc[stop_freq["stop_id"].isin(stop_list_str)].sort_values(by='stop_id').to_excel("PTN_stop_frequency.xlsx")
# type(stop_freq["stop_id"][1])
# stop_freq

In [140]:
line_freq = feed.lines_freq

INFO:root:Reading "shapes.txt".
/Users/sulemanbasit/Library/Python/3.9/lib/python/site-packages/gtfs_functions/gtfs_functions.py:730: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  .apply(lambda x: LineString(zip(x[1], x[0])), axis=1)
/Users/sulemanbasit/Library/Python/3.9/lib/python/site-packages/geopandas/geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [142]:
line_freq.loc[line_freq["route_name"].str.contains("Line|MAX")].sort_values(by=['window']).to_excel("Lines_Frequency.xlsx")

In [147]:
feed.routes_patterns

,route_id,route_name,pattern_id,direction_id,shape_id,zipped_stops,cnt_trips,route_pattern,pattern_name
0,1-20727,1 Bowness/Forest Lawn,82808e0679b57a6728,0,10167,"[('5027', 1), ('8805', 2), ('6739', 3), ('5028...",45,A,1 Bowness/Forest Lawn - 0 - A
1,1-20727,1 Bowness/Forest Lawn,8a9d53b816411eb4ff,1,10160,"[('5067', 1), ('5069', 2), ('5070', 3), ('8863...",44,A,1 Bowness/Forest Lawn - 1 - A
2,10-20727,10 City Hall/Southcentre,dfb87b2271eef189ee,0,100114,"[('8434', 1), ('8392', 2), ('5957', 3), ('7961...",30,A,10 City Hall/Southcentre - 0 - A
3,10-20727,10 City Hall/Southcentre,068d16e4f98c7813fa,1,100120,"[('7380', 1), ('5284', 2), ('5285', 3), ('7930...",28,A,10 City Hall/Southcentre - 1 - A
4,100-20727,100 Airport,9aae9489237977bc83,0,1000073,"[('3497', 1), ('4097', 2), ('4098', 3), ('7448...",39,A,100 Airport - 0 - A
...,...,...,...,...,...,...,...,...,...
247,97-20727,97 South Ranchlands/Scenic Acres,9bcb92d6d9141a8ebc,1,970015,"[('4002', 1), ('7400', 2), ('5476', 3), ('7107...",27,A,97 South Ranchlands/Scenic Acres - 1 - A
248,98-20727,98 Cougar Ridge,c971a99e5a07daa765,0,980155,"[('8374', 1), ('7996', 2), ('9686', 3), ('9688...",30,A,98 Cougar Ridge - 0 - A
249,98-20727,98 Cougar Ridge,4079404a8e8802f457,1,980156,"[('8822', 1), ('9358', 2), ('7311', 3), ('7291...",30,A,98 Cougar Ridge - 1 - A
250,99-20727,99 Acadia/Oakridge,2049a8eaa88d2d998c,0,990039,"[('6440', 1), ('6173', 2), ('6113', 3), ('6441...",35,A,99 Acadia/Oakridge - 0 - A


In [153]:
seg = feed.segments

In [161]:
seg.loc[seg["route_name"].str.contains("MAX|Line")]

,shape_id,route_id,route_name,direction_id,stop_sequence,segment_name,start_stop_name,end_stop_name,segment_id,start_stop_id,end_stop_id,distance_m,geometry
2737,2010471,201-20718,201 Red Line - Somerset - Bridlewood/Tuscany C...,1,1,SB Canyon Meadows CTrain Station - SB Fish Cre...,SB Canyon Meadows CTrain Station,SB Fish Creek - Lacombe CTrain Station,9264 - 9387,9264,9387,1453.803723,"LINESTRING (-114.06988 50.93640, -114.07002 50..."
2738,2010471,201-20718,201 Red Line - Somerset - Bridlewood/Tuscany C...,1,2,SB Fish Creek - Lacombe CTrain Station - SB Sh...,SB Fish Creek - Lacombe CTrain Station,SB Shawnessy CTrain Station,9387 - 9392,9387,9392,1466.709490,"LINESTRING (-114.07336 50.92373, -114.07335 50..."
2739,2010471,201-20718,201 Red Line - Somerset - Bridlewood/Tuscany C...,1,3,SB Shawnessy CTrain Station - SB Somerset - Br...,SB Shawnessy CTrain Station,SB Somerset - Bridlewood CTrain Station,9392 - 9386,9392,9386,1347.041605,"LINESTRING (-114.07061 50.91074, -114.07062 50..."
2740,2010472,201-20718,201 Red Line - Somerset - Bridlewood/Tuscany C...,0,1,NB Somerset - Bridlewood CTrain Station - NB S...,NB Somerset - Bridlewood CTrain Station,NB Shawnessy CTrain Station,9390 - 9391,9390,9391,1353.388799,"LINESTRING (-114.06873 50.89875, -114.06873 50..."
2741,2010472,201-20718,201 Red Line - Somerset - Bridlewood/Tuscany C...,0,2,NB Shawnessy CTrain Station - NB Fish Creek - ...,NB Shawnessy CTrain Station,NB Fish Creek - Lacombe CTrain Station,9391 - 9261,9391,9261,1455.682579,"LINESTRING (-114.07055 50.91077, -114.07053 50..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3896,3070058,307-20727,307 MAX Purple City Centre/East Hills,0,10,36th Street SE Station (EB) - 44th Street SE S...,36th Street SE Station (EB),44th Street SE Station (EB),9808 - 9809,9808,9809,804.739451,"LINESTRING (-113.98073 51.03778, -113.98073 51..."
3897,3070058,307-20727,307 MAX Purple City Centre/East Hills,0,11,44th Street SE Station (EB) - 52nd Street SE S...,44th Street SE Station (EB),52nd Street SE Station (EB),9809 - 9810,9809,9810,823.870224,"LINESTRING (-113.96929 51.03783, -113.96924 51..."
3898,3070058,307-20727,307 MAX Purple City Centre/East Hills,0,12,52nd Street SE Station (EB) - EB 17th AV SE @ ...,52nd Street SE Station (EB),EB 17th AV SE @ 61st ST SE,9810 - 4352,9810,4352,1080.502281,"LINESTRING (-113.95755 51.03785, -113.95752 51..."
3899,3070058,307-20727,307 MAX Purple City Centre/East Hills,0,13,EB 17th AV SE @ 61st ST SE - EB 17th AV SE @ 7...,EB 17th AV SE @ 61st ST SE,EB 17th AV SE @ 70th ST SE,4352 - 2178,4352,2178,878.339223,"LINESTRING (-113.94216 51.03780, -113.94216 51..."


In [166]:
feed = Feed(gtfs_path,start_date="2024-02-17", end_date="2024-02-17")

feed.stop_times.fillna('', inplace=True)
feed.trips.fillna('', inplace=True)
# Repeat for other DataFrame attributes as needed

# Now call get_segments_freq()
segments_freq = feed.segments_freq

INFO:root:Reading "stop_times.txt".
INFO:root:get trips in stop_times
INFO:root:accessing trips
INFO:root:Reading "routes.txt".
INFO:root:Reading "trips.txt".
INFO:root:Reading "calendar.txt".
INFO:root:Reading "calendar_dates.txt".
INFO:root:The busiest date/s of this feed or your selected date range is/are:  ['2024-02-17'] with 6826 trips.
INFO:root:In the case that more than one busiest date was found, the first one will be considered.
INFO:root:In this case is 2024-02-17.
INFO:root:Reading "stop_times.txt".
INFO:root:_trips is defined in stop_times
INFO:root:Reading "stops.txt".
INFO:root:computing patterns
/Users/sulemanbasit/Library/Python/3.9/lib/python/site-packages/gtfs_functions/gtfs_functions.py:380: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

In [176]:
segments_freq.loc[segments_freq["route_name"].str.contains("MAX|Line")].to_excel('PTN segment freq.xlsx')